In [93]:
from tqdm import tqdm
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import time
import random

class JockeyResult:
    @staticmethod
    def scrape(jockey_id_list):
        jockey_results = {}
        for jockey_id in tqdm(jockey_id_list):
            try:
                url = 'https://db.netkeiba.com/jockey/result/' + str(jockey_id) + '/'
                response = requests.get(url)
                response.encoding = 'EUC-JP'
                html = response.text

                df = pd.read_html(html)[0]

                soup = BeautifulSoup(html, 'html.parser')
                text = soup.select('div.db_head_name p')[0].text

                birth_year = int(re.findall(r'\d+', text)[0])
                df['生年'] = [birth_year] * len(df)
                df.index = [jockey_id] * len(df)
    #             old = []
    #             for _, row in df.iterrows():
    #                 if row['年度']['年度'] == '累計':
    #                     old.append(None)
    #                     continue
    #                 old.append(int(row['年度']['年度']) - birth_year)
    #             df['年齢'] = old

    #             experience = []
    #             for index, row in df.iterrows():
    #                 print(row)
                jockey_results[jockey_id] = df
                
                time.sleep(random.uniform(1, 3))
        
            except IndexError:
                continue
            except AttributeError:
                continue
            except Exception as e:
                print(e)
                break
            except:
                break
                
        jockey_results_df = pd.concat([jockey_results[key] for key in jockey_results])
        
        return jockey_results_df
    
race_results = pd.read_pickle('../DATA/race_results_of_5_years.pkl')
jockey_id_series = race_results['騎手id'].unique()
jockey_id_list = []
for jockey_id in jockey_id_series:
    jockey_id_list.append(str(jockey_id).zfill(5))

jockey_results = JockeyResult.scrape(jockey_id_list)
jockey_results.to_pickle('../DATA/jockey_results_of_5_years.pkl')


100%|█████████████████████████████████████████| 283/283 [10:33<00:00,  2.24s/it]


In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime as dt
from dateutil.relativedelta import relativedelta

def get_jockey_data(jockey_id: str):
    url = 'https://db.netkeiba.com/jockey/result/' + jockey_id + '/'
    response = requests.get(url)
    response.encoding = 'EUC-JP'
    html = response.text

    df = pd.read_html(html)[0]

    soup = BeautifulSoup(html, 'html.parser')
    text = soup.select('div.db_head_name p')[0].text

    birth_year = re.findall(r'\d+', text)[0]
    df['生年'] = [birth_year] * len(df)
    df.index = [jockey_id] * len(df)
    return df['1着']['1着']

print(get_jockey_data('01091'))
# for index, row in df.iterrows():
    
# old = []
# for _, row in df.iterrows():
#     if row['年度']['年度'] == '累計':
#         old.append(None)
#         continue
#     old.append(int(row['年度']['年度']) - birth_year)
# df['年齢'] = old

KeyError: 0

In [16]:
import pandas as pd
from datetime import datetime as dt
from dateutil.relativedelta import relativedelta

df = pd.read_pickle('../DATA/jockey_results_of_5_years.pkl')

def jockey_recent_score(jockey_df, year):
    recent_score = {}
    for jockey_id, df in jockey_df.groupby(jockey_df.index):
        old = year - df['生年'][0]
        recent_runs = []
        recent_victories = []
        recent_tops = []
        runs = []
        first_year = int(df['年度']['年度'].tail(1)[0])
        year_list = []
        for i in range(first_year, year + 1, 1):
            year_list.append(str(i))
            
        for _, row in df.iterrows():
            # 直近3年分の勝率を計算
            if row['年度'][0] in [str(year), str(year - 1), str(year - 2)]:
                recent_runs.append(row['1着'][0] + row['2着'][0] + row['3着'][0] + row['着外'][0])
                recent_victories.append(row['1着'][0] + row['2着'][0] + row['3着'][0])
                recent_tops.append(row['1着'][0])
            
            if row['年度'][0] in year_list:
                runs.append(row['1着'][0] + row['2着'][0] + row['3着'][0] + row['着外'][0])                
                
        try:
            recent_victory_ratio = sum(recent_victories) / sum(recent_runs)
            recent_top_ratio = sum(recent_tops) /sum(recent_runs)
            runs_history = sum(runs)
            
        except ZeroDivisionError:
            recent_victory_ratio = 0
            recent_top_ratio = 0
            runs_history = sum(runs)
            continue
        except Exception as e:
            print(e)
            break
        except:
            break
            
        recent_score[jockey_id] = {'単勝': round(recent_top_ratio, 3), '複勝': round(recent_victory_ratio, 3), '経験': runs_history, '年齢': old, '騎手id': jockey_id}
    return recent_score

this_year = dt.today().year
recent_scores = jockey_recent_score(df, this_year)
jockey_df_for_db = pd.DataFrame.from_dict(recent_scores, orient='index')
print(jockey_df_for_db)
jockey_df_for_db.to_pickle('../DATA/jockey_df_for_db_20240801.pkl')

          単勝     複勝       経験  年齢   騎手id
00422  0.041  0.163  13708.0  54  00422
00641  0.036  0.153  22053.0  58  00641
00652  0.048  0.190  15141.0  56  00652
00660  0.096  0.284  21296.0  56  00660
00666  0.140  0.366  24588.0  55  00666
...      ...    ...      ...  ..    ...
05659  0.112  0.343    143.0  35  05659
05660  0.070  0.268     71.0  38  05660
05661  0.047  0.198     86.0  37  05661
05662  0.000  0.250      4.0  36  05662
05663  0.047  0.155    129.0  42  05663

[245 rows x 5 columns]
